In [1]:
import h5py

In [2]:
h5 = h5py.File("D:\jupyter\pytorch-LapSRN\data\lapsrn_train.h5", 'r')

In [7]:
h5.get('label_x4')

<HDF5 dataset "label_x4": shape (89856, 1, 64, 64), type "<f4">

In [4]:
h5.get('label_x2')

<HDF5 dataset "label_x2": shape (89856, 1, 64, 64), type "<f4">

In [5]:
h5.get('data')

<HDF5 dataset "data": shape (89856, 1, 32, 32), type "<f4">

In [1]:
from visdom import Visdom
import scipy.io as io
import torch
from torch.autograd import Variable
from PIL import Image
import numpy as np
import time, math
import matplotlib.pyplot as plt
import platform
import os
from models.SRCNN.model import SRCNN
from models.FSRCNN.model import FSRCNN
from models.VDSR.model import VDSR
from models.ESPCN.model import ESPCN
from models.DRRN.model import DRRN
from models.ESPCN.model import ESPCN
from models.DRCN.model import DRCN
import cv2

In [2]:
pred = io.loadmat('D:\jupyter\DRCN_code\DRCN model\sf2\DRCN_sf2.mat')  

In [3]:
params = pred['net']['params'][0][0][0]

In [4]:
params[0][1].shape

(3, 3, 1, 256)

In [5]:
filter1_weight = params[0][1].transpose(3,2,1,0)
filter1_bias = params[1][1].squeeze(1)
filter2_weight = params[2][1].transpose(3,2,1,0)
filter2_bias = params[3][1].squeeze(1)
filter19_weight = params[4][1].transpose(3,2,1,0)
filter19_bias = params[5][1].squeeze(1)

filter20_weight = params[6][1].transpose(2,1,0)
filter20_bias = params[7][1].squeeze(1)
filter_share_weight = params[8][1].transpose(3,2,1,0)
filter_share_data = params[9][1].squeeze(1)

ensemble_weight = params[10][1].transpose(2,1,0)

In [6]:
# detect device
use_cuda = True
print("CUDA Available: ", torch.cuda.is_available())
# device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
device = "cpu"

CUDA Available:  True


In [7]:
model = DRCN(img_channels=1, num_recursions=16)
model

DRCN(
  (fea_in_conv1): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fea_in_conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (recursive_conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (reconstruct_conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (reconstruct_conv2): Conv2d(256, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ensemble): Conv2d(17, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU(inplace=True)
)

In [8]:
for name, parms in model.named_parameters():
    print('-->name:', name, '-->grad_requires:', parms.requires_grad, ' -->grad_value:', parms.grad)

-->name: fea_in_conv1.weight -->grad_requires: True  -->grad_value: None
-->name: fea_in_conv1.bias -->grad_requires: True  -->grad_value: None
-->name: fea_in_conv2.weight -->grad_requires: True  -->grad_value: None
-->name: fea_in_conv2.bias -->grad_requires: True  -->grad_value: None
-->name: recursive_conv.weight -->grad_requires: True  -->grad_value: None
-->name: recursive_conv.bias -->grad_requires: True  -->grad_value: None
-->name: reconstruct_conv1.weight -->grad_requires: True  -->grad_value: None
-->name: reconstruct_conv1.bias -->grad_requires: True  -->grad_value: None
-->name: reconstruct_conv2.weight -->grad_requires: True  -->grad_value: None
-->name: reconstruct_conv2.bias -->grad_requires: True  -->grad_value: None
-->name: ensemble.weight -->grad_requires: True  -->grad_value: None


In [9]:
ensemble_weight

array([[[-0.03156392]],

       [[ 0.08646547]],

       [[ 0.3825569 ]],

       [[-0.00126755]],

       [[-0.07290084]],

       [[ 0.23602751]],

       [[ 0.30233803]],

       [[ 0.05404749]],

       [[ 0.3018273 ]],

       [[-0.07793459]],

       [[ 0.3640599 ]],

       [[-0.42109546]],

       [[ 0.2806825 ]],

       [[-0.24244851]],

       [[ 0.06587348]],

       [[-0.11815578]],

       [[ 0.54195195]]], dtype=float32)

In [10]:
model.fea_in_conv1.weight.data.copy_(torch.from_numpy(filter1_weight))
model.fea_in_conv1.bias.data.copy_(torch.from_numpy(filter1_bias))
model.fea_in_conv2.weight.data.copy_(torch.from_numpy(filter2_weight))
model.fea_in_conv2.bias.data.copy_(torch.from_numpy(filter2_bias))
model.reconstruct_conv1.weight.data.copy_(torch.from_numpy(filter19_weight))
model.reconstruct_conv1.bias.data.copy_(torch.from_numpy(filter19_bias))
model.reconstruct_conv2.weight.data.copy_(torch.from_numpy(filter20_weight).unsqueeze(0))
model.reconstruct_conv2.bias.data.copy_(torch.from_numpy(filter20_bias))
model.recursive_conv.weight.data.copy_(torch.from_numpy(filter_share_weight))
model.recursive_conv.bias.data.copy_(torch.from_numpy(filter_share_data))
model.ensemble.weight.data.copy_(torch.from_numpy(ensemble_weight).unsqueeze(0))

tensor([[[[-0.0316]],

         [[ 0.0865]],

         [[ 0.3826]],

         [[-0.0013]],

         [[-0.0729]],

         [[ 0.2360]],

         [[ 0.3023]],

         [[ 0.0540]],

         [[ 0.3018]],

         [[-0.0779]],

         [[ 0.3641]],

         [[-0.4211]],

         [[ 0.2807]],

         [[-0.2424]],

         [[ 0.0659]],

         [[-0.1182]],

         [[ 0.5420]]]])

In [11]:
state = {
    'net': model.state_dict(),
    'psnr': 0,
    'epoch': 0
}
torch.save(state, "F:\\cache\\checkpoint\\DRCN_pretrain.pth")